# Pràctiques de Nous Usos de la Informàtica

**Nom de les persones del grup:** Mr X & Ms. Z

# Pràctica 2. Recomanadors

La base de dades movielens 1M (http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·licules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online MovieLens (http://www.movielens.org/). 

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')


### Inspecció de les taules

In [2]:
users[:10]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [3]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [7]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [8]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [9]:
ratings.sort_index(by='movie_id')[:5]

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [ ]:
movies[:5]

In [ ]:
ratings[:5]

## 2.1 Exemple: Fent càlculs sobre DataFrames.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**. 

El primer pas a obtenir una única estructura que contingui tota la informació. 

Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció:

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_index(by='user_id')[:10]

La funció ``ix`` ens permet obtenir un subconjunt de files i/o columnes:

In [ ]:
print data.ix[1]
print data.ix[3:4]

Els índexs Boolans ens permeten seleccionar una part de la taula que compleix una condició.

In [ ]:
# comptem quin tant per cent de ratings estan fets per una dona
print (data[data['gender']=='F']['rating'].count())/float(data['rating'].count()), '%'

Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes:

In [ ]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [ ]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [ ]:
ratings_by_title = data.groupby('title').size()
print ratings_by_title

Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [ ]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [ ]:
mean_ratings = mean_ratings.ix[active_titles]
mean_ratings.sort_index()[:10]

Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [ ]:
top_female_ratings = mean_ratings.sort_index(by='F', ascending=False)
top_female_ratings[:10]

Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [ ]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre home si dones:

In [ ]:
sorted_by_diff = mean_ratings.sort_index(by='diff')
sorted_by_diff[:15]

Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [ ]:
sorted_by_diff[::-1][:15]

Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [ ]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.ix[active_titles]
rating_std_by_title.order(ascending=False)[:10]

### EXERCICI 1

+ Donada la taula ``data``, calcula la puntuació mitjana de cada usuari. 

In [ ]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)

# la vostra solució aquí

+ Quina és la pel·lícula més ben puntuada? (Guarda aquest valor en una variable ``string``)

In [ ]:
# la vostra solució aquí

### EXERCICI 2

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.<br> 


In [25]:
# la vostra solució aquí
def top_movie(dataFrame,usr):
    return 0

print top_movie(data,1)

# Construcció d'un recomanador.

El següent codi, donat el conjunt de dades, construeix un conjunt d'entrenament i un conjunt  de test:

In [26]:
import numpy as np

# generem subconjunts de training i test
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = data[grouped.for_testing == False]
movielens_test = data[grouped.for_testing == True]
print movielens_train.shape
print movielens_test.shape

(797758, 11)
(202451, 11)


La següent funció **evaluate**(estimate), donat un conjunt de dades d'entrenament i un conjunt de dades de test ens avalua la precisió d'un sistema de recomanació que li passem per paràmetre. Per a cadascun dels elements del conjunt de test haurem de pronosticar el seu valor i comparar-lo amb el valor real que l'usuari li ha asignat. La mesura que utilizarem per avaluar el sistema és la root-mean-square error (rsme)

In [27]:
# definim una funció per avaluar el resultat de la recomanació.

def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate,test=movielens_test):
    ids_to_estimate = zip(test['user_id'], test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    return compute_rmse(estimated, real)

### EXERCICI 3

Fes l'avaluació d'un a política de recomanació naïf d'un ítem a un usuari basada en assignar a l'ítem la mitja de les recomanacions d'aquell  usuari.

In [28]:
def rec1(user_id, item_id,train=movielens_train):
    #Put your recomender here
    return 1

print 'Error: %s' % evaluate(rec1)

Error: 2.81146006712
